In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
# Read in csv files
users_log_data = pd.read_csv('./user_logs.csv')
pd.options.display.max_columns = None

In [3]:
# describe the data
#users_log_data.describe()

In [4]:
# look at the logs with negative seconds played.
mask = users_log_data.total_secs > 0
users_log_data = users_log_data[mask]
print('done')
# there's about 230K out of 147M records with faulty seconds played data (0.1%).
# I'm going to remove for data cleansing purposes. No heart burn losing these records, given my hunch that user engagment
# will impact churn greatly. If not, I can always add these records back in later.

done


In [5]:
# take a look at the first few rows
#users_log_data.head()

In [6]:
#users_log_data.describe()

In [7]:
# get shapes of datasets - caution this cell takes a while to load.
#print("user_logs.shape: ", users_log_data.shape)

# Create aggregated datasets for downstream EDA

In [8]:
users_log_data_sampled = users_log_data.sample(10000)

In [9]:
# create an aggregated dataframe by finding the average streaming activity by user
#users_log_data_averages = users_log_data_sampled.groupby(['msno'])[['num_25', 'num_50', 'num_75', 
#                                                                'num_985', 'num_100', 'num_unq', 'total_secs']].mean()

In [10]:
# rename dataframe with clearer names
#users_log_data_averages.columns = ['daily_avg_num_25', 'daily_avg_num_50', 'daily_avg_num_75',
#                                   'daily_avg_num_985', 'daily_avg_num_100', 'daily_avg_num_unq',
#                                   'daily_avg_total_secs']

In [11]:
# create an aggregated dataframe by getting the total number of unique songs played and seconds
#users_log_data_totals = users_log_data.groupby(['msno'])[['num_unq', 'total_secs']].sum()

In [12]:
# rename dataframe for clarity
#users_log_data_totals.columns = ['total_num_unq', 'total_total_secs']

In [13]:
# merge the two aggregated dataframe together 
#users_log_grouped = users_log_data_totals.merge(users_log_data_averages, how='left', on='msno')

In [14]:
#users_log_grouped.describe()

In [15]:
# create categorical variable of number of songs played

# if lower than 25th percentile then "low usage" 
print(users_log_data.num_unq.quantile(0.25))

# if higher than 75th percentile then "high usage"
print(users_log_data.num_unq.quantile(0.50))
print(users_log_data.num_unq.quantile(0.75))


8.0
19.0
40.0


In [22]:
# create a list of our conditions
conditions = [
    (users_log_data['num_unq'] <= users_log_data.num_unq.quantile(0.25)),
    ((users_log_data['num_unq'] > users_log_data.num_unq.quantile(0.25)) & 
         (users_log_data['num_unq'] <= users_log_data.num_unq.quantile(0.50))),
    ((users_log_data['num_unq'] > users_log_data.num_unq.quantile(0.50)) & 
         (users_log_data['num_unq'] <= users_log_data.num_unq.quantile(0.75))),
    (users_log_data['num_unq'] > users_log_data.num_unq.quantile(0.75))]

# create a list of the values we want to assign for each condition
values = ['bottom_25', '25_50', '50_75', 'top_75']

# create a new column and use np.select to assign values to it using our lists as arguments
users_log_data['tier'] = np.select(conditions, values)

# Export aggregated dataframe

In [23]:
users_log_data.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,tier
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513.0,0.0,0.0,0.0,0.0,1.0,1.0,280.335,bottom_25
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150709.0,9.0,1.0,0.0,0.0,7.0,11.0,1658.948,25_50
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150105.0,3.0,3.0,0.0,0.0,68.0,36.0,17364.956,50_75
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150306.0,1.0,0.0,1.0,1.0,97.0,27.0,24667.317,50_75
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150501.0,3.0,0.0,0.0,0.0,38.0,38.0,9649.029,50_75


In [ ]:
#users_log_grouped.to_csv(path_or_buf='./user_logs_grouped.csv')
print('done')

In [27]:
users_log_grouped = pd.read_csv('./user_logs_grouped.csv')


In [28]:
users_log_grouped

,msno,total_num_unq,total_total_secs,daily_avg_num_25,daily_avg_num_50,daily_avg_num_75,daily_avg_num_985,daily_avg_num_100,daily_avg_num_unq,daily_avg_total_secs
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,121.0,33169.777,3.000000,1.166667,0.500000,1.000000,19.500000,20.166667,5528.296167
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,19063.0,5290185.401,1.223684,0.416667,0.328947,0.267544,94.513158,83.609649,23202.567548
2,+++dz9ZCWE2HB/47pJU82NJXQzQuZDx1Wm50YSk/kKk=,94.0,27344.668,1.500000,0.000000,0.000000,0.500000,52.500000,47.000000,13672.334000
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,964.0,256241.646,5.631579,2.605263,2.868421,4.684211,20.815789,25.368421,6743.201211
4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,5059.0,1354749.777,1.802632,0.706140,0.640351,0.929825,22.530702,22.188596,5941.884987
...,...,...,...,...,...,...,...,...,...,...
3012967,zzz672Xpk1wKox75rJ5gak43ZkFQUV1f7Xek3jnPeRM=,5.0,351.433,0.000000,2.500000,0.000000,0.000000,0.000000,2.500000,175.716500
3012968,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,44.0,4200.106,3.888889,1.000000,0.111111,0.333333,0.888889,4.888889,466.678444
3012969,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,187.0,49413.541,1.444444,0.222222,0.222222,0.444444,21.666667,20.777778,5490.393444
3012970,zzztPAN9xjMytpZ0RN2gU9mScDULJnHQZK8eZb4uELU=,65.0,6624.585,1.842105,0.842105,0.368421,0.210526,0.631579,3.421053,348.662368


In [29]:
# create a list of our conditions
conditions = [
    (users_log_grouped['total_num_unq'] <= users_log_grouped.total_num_unq.quantile(0.25)),
    ((users_log_grouped['total_num_unq'] > users_log_grouped.total_num_unq.quantile(0.25)) & 
         (users_log_grouped['total_num_unq'] <= users_log_grouped.total_num_unq.quantile(0.50))),
    ((users_log_grouped['total_num_unq'] > users_log_grouped.total_num_unq.quantile(0.50)) & 
         (users_log_grouped['total_num_unq'] <= users_log_grouped.total_num_unq.quantile(0.75))),
    (users_log_grouped['total_num_unq'] > users_log_grouped.total_num_unq.quantile(0.75))]

# create a list of the values we want to assign for each condition
values = ['bottom_25', '25_50', '50_75', 'top_75']

# create a new column and use np.select to assign values to it using our lists as arguments
users_log_grouped['tier'] = np.select(conditions, values)

In [30]:
users_log_grouped

,msno,total_num_unq,total_total_secs,daily_avg_num_25,daily_avg_num_50,daily_avg_num_75,daily_avg_num_985,daily_avg_num_100,daily_avg_num_unq,daily_avg_total_secs,tier
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,121.0,33169.777,3.000000,1.166667,0.500000,1.000000,19.500000,20.166667,5528.296167,50_75
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,19063.0,5290185.401,1.223684,0.416667,0.328947,0.267544,94.513158,83.609649,23202.567548,top_75
2,+++dz9ZCWE2HB/47pJU82NJXQzQuZDx1Wm50YSk/kKk=,94.0,27344.668,1.500000,0.000000,0.000000,0.500000,52.500000,47.000000,13672.334000,25_50
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,964.0,256241.646,5.631579,2.605263,2.868421,4.684211,20.815789,25.368421,6743.201211,50_75
4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,5059.0,1354749.777,1.802632,0.706140,0.640351,0.929825,22.530702,22.188596,5941.884987,top_75
...,...,...,...,...,...,...,...,...,...,...,...
3012967,zzz672Xpk1wKox75rJ5gak43ZkFQUV1f7Xek3jnPeRM=,5.0,351.433,0.000000,2.500000,0.000000,0.000000,0.000000,2.500000,175.716500,bottom_25
3012968,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,44.0,4200.106,3.888889,1.000000,0.111111,0.333333,0.888889,4.888889,466.678444,25_50
3012969,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,187.0,49413.541,1.444444,0.222222,0.222222,0.444444,21.666667,20.777778,5490.393444,50_75
3012970,zzztPAN9xjMytpZ0RN2gU9mScDULJnHQZK8eZb4uELU=,65.0,6624.585,1.842105,0.842105,0.368421,0.210526,0.631579,3.421053,348.662368,25_50


In [31]:
users_log_grouped.to_csv(path_or_buf='./user_logs_grouped.csv')
print('done')

done
